In [1]:
import numpy as np
import matplotlib.pyplot as plt
from methods import step_function, gaussian_function, create_step_domain, create_gaussian_domain

In [2]:
def df(domain, dx, D):
    return D * (1/dx**2) * (np.roll(domain, -1) - 2*domain + np.roll(domain, 1))

def explicit_euler(domain, dx, dt, D):
    return domain + dt * df(domain, dx, D)

def rk2(domain, dx, dt, D):
    k1 = df(domain, dx, D)
    k2 = df(domain + k1 * dt, dx, D)
    
    return domain + (k1 + k2) * 0.5 * dt

def rk4(domain, dx, dt, D):
    k1 = df(domain, dx, D)
    k2 = df(domain + k1 * dt * 0.5, dx, D)
    k3 = df(domain + k2 * dt * 0.5, dx, D)
    k4 = df(domain + k3 * dt, dx, D)
    
    return domain + 1/6 * (k1 + 2*k2 + 2*k3 + k4) * dt

In [3]:
# we want to compare two different domains and calculate the median squared deviation
# we the two domains to have the same resolution
def compare_domains(d1, d2):
    median_squared = (d1 - d2)**2
    return np.average(median_squared)
    

In [4]:
d_values = [10**(-3), 10**(-2), 10**(-1), 10**(0)]

N = 100
dx = 1/N
D = 0.1
dt = dx**2 / 2*D

methods = [
    ['RK2', rk2, '-'],
    ['Euler', explicit_euler, ':'],
]
initial_condition = [create_step_domain, create_gaussian_domain]

In [5]:
if(False): # generate plots for different diffusion coefficients
    N = 1000
    dx = 1/N
    num_oscillations = 1000

    # we want to create a plot, where we combine two different 
    x_domain, _ = create_gaussian_domain(N)
    d_values = [0.001, 0.01, 0.1, 1]



    ## here we evaluate the effect, which different D values have on our stuff

    fig, ax = plt.subplots(4, 2, figsize=(12, 18))
    for i in range(len(d_values)):
        for j in range(len(initial_condition)):
            _, domain = initial_condition[j](N)
            ax[i,j].plot(x_domain, domain, label='Initial Condition')
            
            for name, method, ls in methods:
                _, domain = initial_condition[j](N)
                for _ in range(num_oscillations):
                    dt = dx**2 / (d_values[i] * 2)
                    domain = method(domain, dx, dt, d_values[i])
                ax[i, j].plot(x_domain, domain, label=name, linestyle=ls)
            ax[i,j].legend()
            ax[i,j].set_title(f'D = {d_values[i]}')
            
    plt.legend()
    plt.tight_layout()
    plt.savefig('output/exercise_2/different_diffusion_values.pdf', dpi=750)

In [6]:
if(False): # here we generate different plots for different time steps
    N = 100
    x_domain, _ = create_gaussian_domain(N)
    D = 0.05
    base_value = dx**2 / (2 * D)
    time_steps = [0.1, 0.5, 1, 1.01, 1.5]

    num_oscillations = 100
    fig, ax = plt.subplots(len(time_steps), len(initial_condition), figsize=(12, 18))

    for i in range(len(time_steps)):
        for j in range(len(initial_condition)):
            _, domain = initial_condition[j](N)
            ax[i, j].plot(x_domain, domain, label='Initial Condition')
            
            
            for name, method, ls in methods:
                _, domain = initial_condition[j](N)
                for _ in range(num_oscillations):
                    domain = method(domain, dx, time_steps[i] * base_value, D)
                ax[i,j].plot(x_domain, domain, label=name, linestyle=ls)
            ax[i, j].legend()
            ax[i, j].set_title(f'dt = {time_steps[i]} * base_value')
            
    plt.legend()
    plt.tight_layout()
    plt.savefig('output/exercise_2/different_dt_values.pdf', dpi=750)

In [7]:
if(False): # generate plots for different resolutions
    num_oscillations = 10000
    max_time = 0.1
    n_values = [1e1, 1e2, 1e3, 1e4]

    fig, ax = plt.subplots(4, 2, figsize=(12, 18))
    for i in range(len(n_values)):
        
        # calculate the constants
        N = int(n_values[i])
        dx = 1/N
        D = 0.05
        dt = dx**2 / (2 * D)
        
        print(f"Now computing a resolution of {N}")

        for j in range(len(initial_condition)):
            print(f"- using initial condition {j}")
            # compute initial condition once and keep a clean copy
            x_domain, domain0 = initial_condition[j](N)
            ax[i, j].plot(x_domain, domain0, label='Initial Condition')

            # number of iterations for this small-time test
            num_iterations = int(max_time / dt)

            for name, method, ls in methods:
                print(f'- - using method {name} | num_iterations = {num_iterations}')
                # start from the same initial condition for every method
                domain = domain0.copy()

                for a in range(num_iterations):
                    domain = method(domain, dx, dt, D)
                    pass

                ax[i, j].plot(x_domain, domain, label=name, linestyle=ls)
            ax[i,j].legend()
            ax[i,j].set_title(f'N = {N} | {name}')
        
    plt.legend()
    plt.tight_layout()
    plt.savefig('output/exercise_2/different_N_values.pdf', dpi=750)


In [ ]:
# Advection methods
def dv_advection(domain, dx, v):
    """Compute advection derivative using centered difference scheme"""
    return -v * (np.roll(domain, -1) - np.roll(domain, 1)) / (2 * dx)

def explicit_euler_advection(domain, dx, dt, v):
    """Explicit Euler for advection"""
    return domain + dt * dv_advection(domain, dx, v)

def rk2_advection(domain, dx, dt, v):
    """RK2 for advection"""
    k1 = dv_advection(domain, dx, v)
    k2 = dv_advection(domain + k1 * dt, dx, v)
    return domain + (k1 + k2) * 0.5 * dt

def rk4_advection(domain, dx, dt, v):
    """RK4 for advection"""
    k1 = dv_advection(domain, dx, v)
    k2 = dv_advection(domain + k1 * dt * 0.5, dx, v)
    k3 = dv_advection(domain + k2 * dt * 0.5, dx, v)
    k4 = dv_advection(domain + k3 * dt, dx, v)
    return domain + 1/6 * (k1 + 2*k2 + 2*k3 + k4) * dt

# Comparison of advection vs diffusion for different v0 values
v0_values = [0.1, 0.5, 1.0, 2.0]
D = 0.05
max_time = 0.1

# Resolution analysis
n_values = [50, 100, 500, 1000]

fig, ax = plt.subplots(len(v0_values), len(initial_condition), figsize=(14, 16))

print("Starting advection vs diffusion comparison analysis...")

for i, v0 in enumerate(v0_values):
    for j in range(len(initial_condition)):
        print(f"Processing v0 = {v0}, initial condition {j}")
        
        # Use a reference resolution (N=1000) for smooth plotting
        N_ref = 1000
        dx_ref = 1 / N_ref
        dt_ref = min(dx_ref / (2 * v0), dx_ref**2 / (2 * D))  # CFL stability
        num_iterations_ref = int(max_time / dt_ref)
        
        x_domain_ref, domain_ref0 = initial_condition[j](N_ref)
        
        # Plot initial condition
        ax[i, j].plot(x_domain_ref, domain_ref0, label='Initial', linewidth=2, color='black')
        
        # Advection at reference resolution
        domain_advection = domain_ref0.copy()
        for _ in range(num_iterations_ref):
            domain_advection = rk2_advection(domain_advection, dx_ref, dt_ref, v0)
        ax[i, j].plot(x_domain_ref, domain_advection, label='Advection (RK2)', linestyle='-', linewidth=1.5)
        
        # Diffusion at reference resolution
        domain_diffusion = domain_ref0.copy()
        for _ in range(num_iterations_ref):
            domain_diffusion = rk2(domain_diffusion, dx_ref, dt_ref, D)
        ax[i, j].plot(x_domain_ref, domain_diffusion, label='Diffusion (RK2)', linestyle='--', linewidth=1.5)
        
        # Compare across different resolutions for advection
        deviations_advection = []
        deviations_diffusion = []
        resolutions = []
        
        for N in n_values:
            dx = 1 / N
            dt = min(dx / (2 * v0), dx**2 / (2 * D))
            num_iterations = int(max_time / dt)
            
            x_domain, domain0 = initial_condition[j](N)
            
            # Advection
            domain_adv_N = domain0.copy()
            for _ in range(num_iterations):
                domain_adv_N = rk2_advection(domain_adv_N, dx, dt, v0)
            
            # Interpolate to reference resolution for comparison
            domain_adv_interp = np.interp(x_domain_ref, x_domain, domain_adv_N)
            dev_adv = compare_domains(domain_advection, domain_adv_interp)
            deviations_advection.append(dev_adv)
            
            # Diffusion
            domain_diff_N = domain0.copy()
            for _ in range(num_iterations):
                domain_diff_N = rk2(domain_diff_N, dx, dt, D)
            
            # Interpolate to reference resolution for comparison
            domain_diff_interp = np.interp(x_domain_ref, x_domain, domain_diff_N)
            dev_diff = compare_domains(domain_diffusion, domain_diff_interp)
            deviations_diffusion.append(dev_diff)
            
            resolutions.append(N)
        
        ax[i, j].set_title(f'v0 = {v0}, IC = {["Step", "Gaussian"][j]} | MSE: Adv={deviations_advection[1]:.4e}, Diff={deviations_diffusion[1]:.4e}')
        ax[i, j].legend(loc='best', fontsize=8)
        ax[i, j].set_xlabel('x')
        ax[i, j].set_ylabel('u')
        ax[i, j].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('output/exercise_2/advection_vs_diffusion_comparison.pdf', dpi=150)
print("Plot saved to output/exercise_2/advection_vs_diffusion_comparison.pdf")
plt.close()

# Create a separate plot for resolution convergence
fig, ax = plt.subplots(len(v0_values), 2, figsize=(12, 14))

print("Creating resolution convergence analysis...")

for i, v0 in enumerate(v0_values):
    # Test with step function
    print(f"  Resolution analysis for v0 = {v0}")
    
    deviations_adv_step = []
    deviations_diff_step = []
    resolutions = []
    
    N_ref = 2000
    dx_ref = 1 / N_ref
    dt_ref = min(dx_ref / (2 * v0), dx_ref**2 / (2 * D))
    num_iterations_ref = int(max_time / dt_ref)
    
    x_domain_ref, domain_ref0_step = create_step_domain(N_ref)
    
    # Reference solution
    domain_adv_ref = domain_ref0_step.copy()
    domain_diff_ref = domain_ref0_step.copy()
    for _ in range(num_iterations_ref):
        domain_adv_ref = rk2_advection(domain_adv_ref, dx_ref, dt_ref, v0)
        domain_diff_ref = rk2(domain_diff_ref, dx_ref, dt_ref, D)
    
    for N in n_values:
        dx = 1 / N
        dt = min(dx / (2 * v0), dx_ref**2 / (2 * D))
        num_iterations = int(max_time / dt)
        
        x_domain, domain0_step = create_step_domain(N)
        
        # Advection
        domain_adv = domain0_step.copy()
        for _ in range(num_iterations):
            domain_adv = rk2_advection(domain_adv, dx, dt, v0)
        domain_adv_interp = np.interp(x_domain_ref, x_domain, domain_adv)
        dev_adv = compare_domains(domain_adv_ref, domain_adv_interp)
        deviations_adv_step.append(dev_adv)
        
        # Diffusion
        domain_diff = domain0_step.copy()
        for _ in range(num_iterations):
            domain_diff = rk2(domain_diff, dx, dt, D)
        domain_diff_interp = np.interp(x_domain_ref, x_domain, domain_diff)
        dev_diff = compare_domains(domain_diff_ref, domain_diff_interp)
        deviations_diff_step.append(dev_diff)
        
        resolutions.append(N)
    
    # Plot step function convergence
    ax[i, 0].loglog(resolutions, deviations_adv_step, 'o-', label='Advection', linewidth=2)
    ax[i, 0].loglog(resolutions, deviations_diff_step, 's-', label='Diffusion', linewidth=2)
    ax[i, 0].set_title(f'Step Function, v0 = {v0}')
    ax[i, 0].set_xlabel('N (resolution)')
    ax[i, 0].set_ylabel('MSE vs reference')
    ax[i, 0].legend()
    ax[i, 0].grid(True, which='both', alpha=0.3)
    
    # Gaussian function
    deviations_adv_gauss = []
    deviations_diff_gauss = []
    
    x_domain_ref, domain_ref0_gauss = create_gaussian_domain(N_ref)
    
    # Reference solution
    domain_adv_ref = domain_ref0_gauss.copy()
    domain_diff_ref = domain_ref0_gauss.copy()
    for _ in range(num_iterations_ref):
        domain_adv_ref = rk2_advection(domain_adv_ref, dx_ref, dt_ref, v0)
        domain_diff_ref = rk2(domain_diff_ref, dx_ref, dt_ref, D)
    
    for N in n_values:
        dx = 1 / N
        dt = min(dx / (2 * v0), dx_ref**2 / (2 * D))
        num_iterations = int(max_time / dt)
        
        x_domain, domain0_gauss = create_gaussian_domain(N)
        
        # Advection
        domain_adv = domain0_gauss.copy()
        for _ in range(num_iterations):
            domain_adv = rk2_advection(domain_adv, dx, dt, v0)
        domain_adv_interp = np.interp(x_domain_ref, x_domain, domain_adv)
        dev_adv = compare_domains(domain_adv_ref, domain_adv_interp)
        deviations_adv_gauss.append(dev_adv)
        
        # Diffusion
        domain_diff = domain0_gauss.copy()
        for _ in range(num_iterations):
            domain_diff = rk2(domain_diff, dx, dt, D)
        domain_diff_interp = np.interp(x_domain_ref, x_domain, domain_diff)
        dev_diff = compare_domains(domain_diff_ref, domain_diff_interp)
        deviations_diff_gauss.append(dev_diff)
    
    # Plot gaussian convergence
    ax[i, 1].loglog(resolutions, deviations_adv_gauss, 'o-', label='Advection', linewidth=2)
    ax[i, 1].loglog(resolutions, deviations_diff_gauss, 's-', label='Diffusion', linewidth=2)
    ax[i, 1].set_title(f'Gaussian Function, v0 = {v0}')
    ax[i, 1].set_xlabel('N (resolution)')
    ax[i, 1].set_ylabel('MSE vs reference')
    ax[i, 1].legend()
    ax[i, 1].grid(True, which='both', alpha=0.3)

plt.tight_layout()
plt.savefig('output/exercise_2/resolution_convergence.pdf', dpi=150)
print("Convergence plot saved to output/exercise_2/resolution_convergence.pdf")
plt.close()

print("Analysis complete!")